In [10]:
import importlib

import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt
from tensorflow.keras import Input, Model, layers, callbacks
from sklearn.model_selection import train_test_split

import utils
importlib.reload(utils)

from utils import prepare_dataframe, df_to_np_arrays

In [4]:
tf.keras.backend.clear_session()

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
#DATASET_DIR = "datasets/utkface/"
DATASET_DIR = "UTKFace/"
IMG_DIM = 200
INPUT_SHAPE = (IMG_DIM, IMG_DIM, 3)

FIXED_KERNEL_SIZE = (3, 3)
MIN_DENSE_UNITS = 16
NUM_AGE_CLASSES = 12

In [7]:
df = prepare_dataframe(DATASET_DIR)

In [11]:
# Define the bin edges
bins = list(range(0, 81, 10)) + [df['Age'].max() + 1]  # This adds a final upper bound for 80+

# Create bin labels (optional)
labels = [f"{i}-{i+9}" for i in range(0, 80, 10)] + ['80+']

# Apply binning
df['Age_Bin'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False, include_lowest=True)
df

,Image_Path,Age,Gender,Ethnicity,Age_Class,Age_Bin
0,UTKFace/100_0_0_20170112213500903.jpg.chip.jpg,100,0,0,10,80+
1,UTKFace/100_0_0_20170112215240346.jpg.chip.jpg,100,0,0,10,80+
2,UTKFace/100_1_0_20170110183726390.jpg.chip.jpg,100,1,0,10,80+
3,UTKFace/100_1_0_20170112213001988.jpg.chip.jpg,100,1,0,10,80+
4,UTKFace/100_1_0_20170112213303693.jpg.chip.jpg,100,1,0,10,80+
...,...,...,...,...,...,...
23700,UTKFace/9_1_3_20161220222856346.jpg.chip.jpg,9,1,3,0,0-9
23701,UTKFace/9_1_3_20170104222949455.jpg.chip.jpg,9,1,3,0,0-9
23702,UTKFace/9_1_4_20170103200637399.jpg.chip.jpg,9,1,4,0,0-9
23703,UTKFace/9_1_4_20170103200814791.jpg.chip.jpg,9,1,4,0,0-9


In [12]:
# Train-test split, used for evaluating final model

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
X_train_cv, y_train_cv = df_to_np_arrays(df_train, IMG_DIM)
X_test, y_test = df_to_np_arrays(df_test, IMG_DIM)

KeyboardInterrupt: 

In [ ]:
# Train-val split, used for evaluating intermediate model

df_train_tune, df_val_tune = train_test_split(df_train, test_size=0.2, random_state=42)
X_train_tune, y_train_tune = df_to_np_arrays(df_train_tune)
X_val, y_val = df_to_np_arrays(df_val_tune)


NameError: name 'train_test_split' is not defined

In [ ]:
# Train-val split, used for evaluating intermediate model

df_train_tune, df_val_tune = train_test_split(df_train, test_size=0.2, random_state=42)

max_count = df_train_tune['Age_Bin'].value_counts().max()

df_balanced_age = df_train_tune.groupby('Age_Bin').apply(
    lambda x: x.sample(max_count, replace=True, random_state=1)
    ).reset_index(drop=True)

max_count = df_balanced_age['Ethnicity'].value_counts().max()

df_balanced_age_race = df_balanced_age.groupby('Ethnicity').apply(
    lambda x: x.sample(max_count, replace=True, random_state=1)
    ).reset_index(drop=True)


X_train_tune, y_train_tune = df_to_np_arrays(df_balanced_age_race)
X_val, y_val = df_to_np_arrays(df_val_tune)

In [ ]:
# Based on existing research found here: https://www.kaggle.com/datasets/jangedoo/utkface-new/code

class AgeHyperModel(kt.HyperModel):
    def build(self, hp):
        inputs = Input(shape=INPUT_SHAPE, dtype=tf.float32, name="input_image")
        
        hp_use_batch_norm = hp.Boolean("use_batch_norm_global", default=False)
        hp_dropout_rate = hp.Float("dropout_rate_global", min_value=0.1, max_value=0.5, step=0.1, default=0.25)
        hp_num_conv_blocks = hp.Int("num_conv_blocks", min_value=1, max_value=4, step=1, default=3)
        
        current_filters = 0  # Tracks filters in the latest conv layer.
        x = inputs

        for i in range(hp_num_conv_blocks):
            if i == 0:
                # First conv block: choice between 32 and 64 filters.
                current_filters = hp.Choice("filters_start", values=[32, 64], default=32)
            else:
                # Next blocks: double filters (capped at 512).
                current_filters = min(512, current_filters * 2)

            x = layers.Conv2D(filters=current_filters, kernel_size=FIXED_KERNEL_SIZE, activation=None)(x)

            if hp_use_batch_norm:
                x = layers.BatchNormalization()(x)

            x = layers.Activation("relu")(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        
        last_conv_filters = current_filters
        x = layers.Flatten(name="flatten")(x)

        hp_num_dense_layers = hp.Int("num_dense_layers", min_value=1, max_value=3, step=1, default=2)
        current_dense_units = 0

        for i in range(hp_num_dense_layers):
            hp_size_choice = hp.Choice("size_choice", values=["same", "half"], default="half")

            if hp_size_choice == "same":
                current_dense_units = max(MIN_DENSE_UNITS, last_conv_filters)
            else:
                current_dense_units = max(MIN_DENSE_UNITS, last_conv_filters // 2)

            x = layers.Dense(units=current_dense_units, activation="relu")(x)
            x = layers.Dropout(rate=hp_dropout_rate)(x)
        
        outputs = layers.Dense(NUM_AGE_CLASSES, activation="softmax", name="age_class_output")(x)
        model = Model(inputs=inputs, outputs=outputs)
        
        hp_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log", default=1e-3)
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
        model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

        return model

    # Idea taken from: https://github.com/keras-team/keras-tuner/issues/122#issuecomment-544648268
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args, batch_size=hp.Choice("batch_size", values=[32, 64, 128, 256], default=64), **kwargs,)

In [89]:
tuner = kt.Hyperband(
    AgeHyperModel(),
    objective="val_accuracy",
    factor=3,
    directory="hyperparameter_tuning",
    project_name="ageclass_tuning",
    # max_trials=15
)

Reloading Tuner from hyperparameter_tuning/ageclass_tuning/tuner0.json


In [ ]:
tuner.search(
    train_dataset_tune,
    validation_data=val_dataset_tune,
    epochs=20,
    callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)